In [5]:
from datetime import datetime
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

# spark.sparkContext.getConf().getAll()

In [32]:
spark.stop()

In [6]:
silver_products = spark.read.format("parquet").load("s3a://warehouse/silver/tiki/products.parquet")

In [7]:
x = df.iloc[0]['authors']
x != []

False

In [12]:
df = silver_products.toPandas()
df[df['authors'].apply(len) > 1]['authors']

307      [(253291, Karen Dillon), (253293, James Allwor...
367      [(286405, Michele Cagan), (286407, Elisabeth L...
368      [(286359, David Carey), (286361, John E. Morris)]
369       [(286333, Blake Masters), (286335, Peter Thiel)]
411      [(3407, Daniel Goleman), (144149, Tasha Eurich...
417      [(1777, Andrea Schulte-Peevers), (1838, Lonely...
434      [(172169, Avinash K. Dixit), (286323, Barry J....
516      [(52495, Susan Cain), (430929, Gregory Mone), ...
531      [(3300, F. Scott Fitzgerald), (428755, Tony Ta...
536       [(3538, Emily Bronte), (431789, Pauline Nestor)]
538      [(48690, Ryan Holiday), (85288, Stephen Hansel...
671          [(515015, Eric Butow), (515017, Jenn Herman)]
972      [(709055, Aleksandra Mizielińska), (709059, Da...
1936     [(457351, Masami Sasaki), (457353, Wakamatsu A...
1937         [(11658, Jo Condrill), (11660, Bennie Bough)]
2144              [(359, Spencer Johnson), (312479, M.D.)]
2698     [(5338645, Kenji Shimizu), (5338649, Suzuki Hi.

In [10]:
from pyspark.sql.functions import col, explode
exploded_authors = silver_products.withColumn("author", explode(col("authors"))) 

In [14]:
exploded_authors.toPandas()['author']

0               (21038, Ibuka Masaru)
1             (63949, Khương Lệ Bình)
2      (1178, Sir Arthur Conan Doyle)
3               (143619, Mark Manson)
4            (149823, Caroline Barry)
                    ...              
631                  (8249934, Fūmin)
632           (25537, Colleen Hoover)
633        (7476405, Chieko Hosokawa)
634                  (8249934, Fūmin)
635          (611193, Timothy Ferris)
Name: author, Length: 636, dtype: object

In [25]:
from pyspark.sql.functions import col, explode, arrays_zip

def transform_silver_products(silver_products):

    select_columns = [
        'category_id', 'product_id', 'seller_id', 'brand_id', 'product_name', 'description', 'specifications',
        'breadcrumbs', 'original_price', 'discount', 'price', 'discount_rate', 'quantity_sold', 'rating_average',
        'review_count', 'day_ago_created', 'product_url', 'is_authentic', 'is_freeship_xtra',
        'is_top_deal', 'return_reason', 'inventory_type', 'warranty_period', 'warranty_type', 'warranty_location'
    ]

    gold_products = silver_products.select(*select_columns)

    gold_images_url = silver_products.select(
        col("product_id"),
        col("seller_id"),
        explode(col("images_url")).alias("image_url")
    )

    exploded_authors = silver_products.withColumn("author", explode(col("authors"))) \
                                      .select(
                                          col("author._1").alias("author_id"),
                                          col("author._2").alias("author_name")
                                      )

    gold_authors = exploded_authors.drop_duplicates(["author_id"]).dropna()

    product_author_pairs = silver_products.withColumn("author", explode(col("authors"))) \
                                          .select(
                                              col("product_id"),
                                              col("seller_id"),
                                              col("author._1").alias("author_id")
                                          )

    gold_products_authors = product_author_pairs.drop_duplicates()

    gold_brands = silver_products.select("brand_id", "brand_name") \
                                 .drop_duplicates(["brand_id"]) \
                                 .dropna()

    return {
        "gold_products": gold_products,
        "gold_images_url": gold_images_url,
        "gold_authors": gold_authors,
        "gold_products_authors": gold_products_authors,
        "gold_brands": gold_brands
    }
result = transform_silver_products(silver_products)

In [31]:
x = result["gold_products_authors"].toPandas()
x['author_id'].value_counts()

author_id
25537     80
46005      7
79888      7
4406       6
12361      6
          ..
232165     1
3300       1
860929     1
19615      1
20806      1
Name: count, Length: 449, dtype: int64

In [47]:
# Register the DataFrame as a temporary SQL view
silver_products.createOrReplaceTempView("silver_products")

# Run SQL query
result = spark.sql("SELECT * FROM silver_products WHERE product_id = 11505582")

# Show the result
result.toPandas()['authors_name']


0    [Daniel Goleman, Morten Hansen, Harvard Busine...
Name: authors_name, dtype: object

In [16]:
# Register the DataFrame as a temporary SQL view
gold_authors.createOrReplaceTempView("gold_authors_view")

# Run SQL query
result = spark.sql("SELECT * FROM gold_authors_view WHERE author_name = 'Daniel Goleman'")

# Show the result
result.show()


+---------+--------------+
|author_id|   author_name|
+---------+--------------+
|     3407|Daniel Goleman|
|   144149|Daniel Goleman|
|   172475|Daniel Goleman|
|   172509|Daniel Goleman|
|   304427|Daniel Goleman|
|   309375|Daniel Goleman|
|   310039|Daniel Goleman|
+---------+--------------+



In [32]:
users = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/users.parquet")

In [3]:
sellers = spark.read.format("parquet").load("s3a://warehouse/silver/tiki/sellers.parquet")

In [5]:
sellers.toPandas()['store_level'].unique()

array(['OFFICIAL_STORE', 'NONE', 'TRUSTED_STORE'], dtype=object)

In [33]:
from pyspark.sql.functions import col, lit, when, from_unixtime, udf, explode
from pyspark.sql.types import IntegerType

In [5]:
products = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/products.parquet")
# products = products.toPandas()
# products.info()


In [23]:
from pyspark.sql.functions import when, col, isnan

products = products.withColumn(
    "warranty_type",
    when(isnan(col("warranty_type")) | col("warranty_type").isNull(), "Không bảo hành")
    .otherwise(col("warranty_type"))
)

products = products.withColumn(
    "warranty_location",
    when(isnan(col("warranty_location")) | col("warranty_location").isNull(), "Không bảo hành")
    .otherwise(col("warranty_location"))
)
products = products.withColumn(
    "return_reason",
    when(col("return_reason") == "any_reason", "Bất cứ lý do gì")
    .when(col("return_reason") == "defective_product", "Sản phẩm hư hỏng")
    .when((col("return_reason") == "no_return") | col("return_reason").isNull(), "Không đổi trả")
    .otherwise(col("return_reason"))
)

# Hiển thị kết quả
products.toPandas()['return_reason'].value_counts()


return_reason
Bất cứ lý do gì     9455
Sản phẩm hư hỏng    1328
Không đổi trả        342
Name: count, dtype: int64

In [4]:
products = products.filsplna("Không bảo hành", subset=["warranty_type", "warranty_location"])
# products_spark_df = products_spark_df.fillna("no_return", subset=["return_reason"])
# products_spark_df = products_spark_df.fillna(0, subset=["quantity_sold"])

TypeError: DataFrame.fillna() got an unexpected keyword argument 'subset'

In [6]:
products_pd = products.toPandas()

# products_pd[products_pd['warranty_type'] == 'NaN']
products_pd['warranty_type'].value_counts()

warranty_type
Không bảo hành    7136
Điện tử           1528
Hóa đơn           1291
Phiếu bảo hành     774
Tem bảo hành       396
Name: count, dtype: int64

In [4]:
img_url = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/images_url.parquet")
img_url = img_url.toPandas()
img_url.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69309 entries, 0 to 69308
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  69309 non-null  int64 
 1   seller_id   69309 non-null  int64 
 2   image_url   69309 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.6+ MB


In [5]:
img_url.head()

,product_id,seller_id,image_url
0,122012,1,https://salt.tikicdn.com/ts/product/24/85/1a/7...
1,122012,1,https://salt.tikicdn.com/media/catalog/product...
2,122012,1,https://salt.tikicdn.com/media/catalog/product...
3,122012,1,https://salt.tikicdn.com/media/catalog/product...
4,122012,1,https://salt.tikicdn.com/media/catalog/product...


In [35]:
users = users.toPandas()
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133909 entries, 0 to 133908
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       133909 non-null  int64  
 1   user_name     133054 non-null  object 
 2   avatar_url    133909 non-null  object 
 3   joined_day    133909 non-null  int32  
 4   joined_time   133869 non-null  object 
 5   total_review  133869 non-null  float64
 6   total_thank   133869 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(3)
memory usage: 6.6+ MB


In [36]:
users['avatar_url'].value_counts()

avatar_url
//tiki.vn/assets/img/avatar.png                                                                                                                                                                                                                                       97664
http://s120.avatar.talk.zdn.vn/default                                                                                                                                                                                                                                   17
https://s120.avatar.talk.zdn.vn/default                                                                                                                                                                                                                                  11
https://scontent-iad3-1.xx.fbcdn.net/v/t31.0-1/c59.0.200.200a/p200x200/10733713_10150004552801937_4553731092814901385_o.jpg?_nc_cat=1&_nc_sid=12b3be&_nc_ohc=kbRb-3obZ7MAX9TZwXo&_nc_ht=s